In [1]:
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import spacy

In [2]:
nlp = spacy.load("en_core_web_sm")  # for NER

In [3]:
class CHILL_NLP:
    def __init__(self, training_questions, training_labels):
        self.query_templates = {
            "SELECT": "SELECT {columns} FROM {table} WHERE {condition};",
            "COUNT": "SELECT COUNT(*) FROM {table} WHERE {condition};",
            "MAX": "SELECT MAX({column}) FROM {table} WHERE {condition};",
            "MIN": "SELECT MIN({column}) FROM {table} WHERE {condition};",
            "AVG": "SELECT AVG({column}) FROM {table} WHERE {condition};",
            "SUM": "SELECT SUM({column}) FROM {table} WHERE {condition};"
        }
        self.nlp = spacy.load("en_core_web_sm")

        # Dictionary to classify locations
        self.location_types = {
            "Quezon City": "city",
            "Manila City": "city",
            "Cebu City": "city",
            "NCR": "region",
            "Luzon": "island",
            "Visayas": "island",
            "Mindanao": "island",
            "Palawan": "province",
            "Cebu": "province",
            "Davao": "province",
            "Bulacan": "province",
        }

    def process_question(self, question):
        """Tokenizes and POS tags the input question."""
        tokens = word_tokenize(question)
        tagged = pos_tag(tokens)
        return tokens, tagged

    def extract_location(self, question):
        """Uses spaCy's Named Entity Recognition (NER) for better location extraction."""
        doc = self.nlp(question)
        locations = [ent.text for ent in doc.ents if ent.label_ in ["GPE", "LOC"]]
        return locations[0] if locations else None  # Return first detected location

    def get_location_condition(self, location):
        """Determines the correct SQL condition based on location type."""
        if location in self.location_types:
            location_type = self.location_types[location]
            return f"{location_type} = '{location}'"
        return None

    def parse_question(self, question):
        """Determines SQL intent and extracts query components."""
        tokens, tagged = self.process_question(question)
        table, column, condition = "philippines_db", None, None  # Default condition as None

        # Extract location
        location = self.extract_location(question)
        if location:
            condition = self.get_location_condition(location)

        # Identify query type
        intent = "SELECT"  # Default intent

        if "population" in tokens:
            column = "population"
        elif "capital" in tokens:
            column = "capital"
        elif "barangay" in tokens or "barangays" in tokens:
            column = "barangay_count"
            intent = "COUNT"
        elif "area" in tokens:
            column = "land_area"
        elif "literacy" in tokens:
            column = "literacy_rate"
        elif "average" in tokens:
            column = "some_numeric_column"  # Default for averaging
            intent = "AVG"
        elif "total" in tokens or "sum" in tokens:
            column = "some_numeric_column"  # Default for summing
            intent = "SUM"

        # If no valid location is found, set a generic condition
        if condition is None:
            if "city" in tokens or "cities" in tokens:
                condition = "city IS NOT NULL"
            elif "province" in tokens or "provinces" in tokens:
                condition = "province IS NOT NULL"
            elif "region" in tokens or "regions" in tokens:
                condition = "region IS NOT NULL"
            elif "island" in tokens or "islands" in tokens:
                condition = "island IS NOT NULL"
            else:
                condition = "1=1"  # Last fallback

        # Ensure column is properly assigned for non-COUNT queries
        if intent in ["SELECT", "MAX", "MIN", "AVG", "SUM"] and column is None:
            raise KeyError(f"Column not defined for intent '{intent}' in question: '{question}'")

        # Ensure intent exists in query_templates
        if intent not in self.query_templates:
            raise ValueError(f"Unknown intent '{intent}' derived from question: {question}")

        return self.query_templates[intent].format(columns=column, table=table, condition=condition)

    def generate_sql(self, question):
        """Converts a natural language question to an SQL query."""
        return self.parse_question(question)

In [4]:
training_questions = [
    "What is the population of Manila?",
    "How many cities are in Luzon?",
    "Which province has the highest population?",
    "What is the capital of Cebu?",
    "What are the major tourist spots in Palawan?",
    "How many barangays are in Quezon City?",
    "Which city has the largest land area?",
    "What is the literacy rate in Metro Manila?",
    "Which island has the most provinces?",
    "What is the GDP of the Philippines?"
]

training_labels = [
    "SELECT",  # "What is the population of Manila?"
    "COUNT",   # "How many cities are in Luzon?"
    "MAX",     # "Which province has the highest population?"
    "SELECT",  # "What is the capital of Cebu?"
    "SELECT",  # "What are the major tourist spots in Palawan?"
    "COUNT",   # "How many barangays are in Quezon City?"
    "MAX",     # "Which city has the largest land area?"
    "SELECT",  # "What is the literacy rate in Metro Manila?"
    "MAX",     # "Which island has the most provinces?"
    "SELECT"   # "What is the GDP of the Philippines?"
]

In [5]:
chill_nlp = CHILL_NLP(training_questions, training_labels)

In [6]:
# Example Questions
example_questions = [
    "What is the population of Quezon City?",
    "How many barangays are in Manila City?",
    "What is the total land area of Luzon?",
    "What is the capital of Cebu?",
    "What is the literacy rate in Bulacan?"
]

for q in example_questions:
    try:
        sql_query = chill_nlp.generate_sql(q)
        print(f"Question: {q}\nSQL Query: {sql_query}\n")
    except Exception as e:
        print(f"Error processing question '{q}': {e}")

Question: What is the population of Quezon City?
SQL Query: SELECT population FROM philippines_db WHERE city = 'Quezon City';

Question: How many barangays are in Manila City?
SQL Query: SELECT COUNT(*) FROM philippines_db WHERE city = 'Manila City';

Question: What is the total land area of Luzon?
SQL Query: SELECT land_area FROM philippines_db WHERE island = 'Luzon';

Question: What is the capital of Cebu?
SQL Query: SELECT capital FROM philippines_db WHERE province = 'Cebu';

Question: What is the literacy rate in Bulacan?
SQL Query: SELECT literacy_rate FROM philippines_db WHERE province = 'Bulacan';

